In [ ]:
### task
 - считывать картинку + ближайшую точку на соотв disparity map

- выделить маску

### Data description

### Imports

In [1]:
import cv2
import glob
import os
import sys
import numpy as np
from collections import defaultdict
from matplotlib import pyplot as plt
%matplotlib inline


#### Reading

In [3]:
path_data = '/home/anastasia/obj_detection/data/cityscapes/disparity'
print os.path.abspath(path_data)

/home/anastasia/obj_detection/data/cityscapes/disparity


In [4]:
# paths 
cityscapesPath = os.path.normpath(os.path.join(os.path.dirname('__file__'), '..', 'data/cityscapes'))
img_dir = 'leftImg8bit'
path_left_imgs = os.path.normpath(os.path.join(cityscapesPath,img_dir))

# default dictionary of cities
cities = defaultdict(list)
modes = ['train','test','val']
for mode in modes:
    for (dirpath, dirnames, filenames) in os.walk(path_left_imgs+'/'+mode):
        cities[mode] = dirnames 
        break

        
mode = 'train'
city = 'zurich'

# return list of paths to imgs for this city
def read_imgs(mode=mode, city=city):
    
    # absolute path to the directory with images
    path_imgs = os.path.normpath(os.path.join(cityscapesPath,img_dir, mode, city))
    # list of absolute paths to images
    imgs = glob.glob(path_imgs + '/*.png')
    print '%s %s: number of images is' %(mode,city), len(imgs)
    
    return imgs

# return list of paths to disparity imgs for this city
def read_disparities(mode=mode, city=city):
    
    # absolute path to the directory with disparities
    path_disp = os.path.normpath(os.path.join(cityscapesPath,'disparity', mode, city))
    # list of absolute paths to disparities
    disp_imgs = glob.glob(path_disp + '/*.png')
    print '%s %s: number of disparity images is' %(mode,city), len(disp_imgs)
    
    return disp_imgs

def find_disparity(img_path, mode=mode, city=city):
    
    path_disp = os.path.normpath(os.path.join(cityscapesPath,'disparity', mode, city))
    
    # take a tail of img path
    img_name = os.path.split(img_path)[1]
    # remove extention
    img_name = os.path.splitext(img_name)[0]
    # take img number
    img_number = img_name[:-len(img_dir)]
    # abs path to the corresponded disparity
    disp_path = glob.glob(path_disp +'/'+ str(img_number)+'*')[0]
    
    return disp_path
    
def nearest_point(disp_path, mode=mode, city=city):
    
    disp = cv2.imread(disp_path)
    
    return np.min(disp[np.nonzero(disp)])

## Test

In [5]:
imgs = read_imgs()
disp_imgs = read_disparities()

filename = os.path.split(imgs[0])[1]
name = os.path.splitext(filename)[0][:-len(img_dir)]
print name

disp = cv2.imread(find_disparity(imgs[0]))
print np.max(disp[np.nonzero(disp)])

train zurich: number of images is 122
train zurich: number of disparity images is 122
zurich_000111_000019_
119
